<a href="https://colab.research.google.com/github/ItsMeYobs/Data-Mining-Activity/blob/main/Lecture_07_Index_Objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture 7 - Index Objects

`pandas` Index objects are responsible for holding the axis labels and other metadata
(like the axis name or names). Any array or other sequence of labels you use when
constructing a Series or DataFrame is internally converted to an Index:

In [ ]:
import pandas as pd
import numpy as np

obj = pd.Series(range(3), index=['a', 'b', 'c'])

In [ ]:
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [ ]:
index[1:]

Index(['b', 'c'], dtype='object')

Index objects are immutable and thus can’t be modified by the user:

In [ ]:
index[1] = 'd' # TypeError

TypeError: ignored

Immutability makes it safer to share Index objects among data structures:

In [ ]:
labels = pd.Index(np.arange(3))


NameError: ignored

In [ ]:
obj2 = pd.Series([1.5, -2.5, 0])
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [ ]:
obj2.index is labels

False

Some users will not often take advantage of the capabilities provided by indexes, but because some operations will yield results containing indexed data, it’s important to understand how they work.

In addition to being array-like, an Index also behaves like a fixed-size set:

In [ ]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

frame3 = pd.DataFrame(pop, index=[2000,2001,2002])
frame3

NameError: ignored

In [ ]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object')

In [ ]:
'Ohio' in frame3.columns

True

In [ ]:
2003 in frame3.index

False

Unlike Python sets, a pandas Index can contain duplicate labels:

In [ ]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

Selections with duplicate labels will select all occurrences of that label.

## Essential Functionality

This section will walk you through the fundamental mechanics of interacting with the
data contained in a Series or DataFrame.

### Reindexing

An important method on pandas objects is `reindex`, which means to create a new
object with the data conformed to a new index. Consider an example:

In [ ]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling reindex on this Series rearranges the data according to the new index, introducing missing values if any index values were not already present:

In [ ]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

For ordered data like time series, it may be desirable to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as ffill, which forward-fills the values:

In [ ]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [ ]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, `reindex` can alter either the (row) index, columns, or both. When
passed only a sequence, it reindexes the rows in the result:

In [ ]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [ ]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


The columns can be reindexed with the columns keyword:

In [ ]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


As we’ll explore in more detail, you can reindex more succinctly by label-indexing
with loc, and many users prefer to use it exclusively:

## Dropping Entries from an Axis

Dropping one or more entries from an axis is easy if you already have an index array
or list without those entries. As that can require a bit of munging and set logic, the drop method will return a new object with the indicated value or values deleted from
an axis:

In [ ]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [ ]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [ ]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

With DataFrame, index values can be deleted from either axis. To illustrate this, we
first create an example DataFrame:

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])/topics/assembly-project
data

NameError: ignored

Calling drop with a sequence of labels will drop values from the row labels (axis 0):

In [ ]:
data.drop(['Colorado', 'Ohio'])

NameError: ignored

You can drop values from the columns by passing axis=1 or axis='columns':

In [ ]:
data.drop('two', axis=1)

NameError: ignored

In [ ]:
data.drop(['two', 'four'], axis='columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


Many functions, like drop, which modify the `size` or `shape` of a Series or DataFrame,
can manipulate an object in-place without returning a new object:

In [ ]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

Be careful with the inplace, as it destroys any data that is dropped.

## Indexing, Selection and Filtering

`Series` indexing (`obj[...]`) works analogously to `NumPy` array indexing, except you
can use the `Series`’s index values instead of only integers. Here are some examples of
this:

In [ ]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [ ]:
obj['b']

1.0

In [ ]:
obj[1]

1.0

In [ ]:
obj[:2]

a    0.0
b    1.0
dtype: float64

In [ ]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [ ]:
 obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [ ]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Slicing with labels behaves differently than normal Python slicing in that the end‐point is inclusive:

In [ ]:
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

In [ ]:
obj['b':'d']

b    5.0
c    5.0
d    3.0
dtype: float64

Setting using these methods modifies the corresponding section of the `Series`:

In [ ]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

Indexing into a `DataFrame` is for retrieving one or more columns either with a single
value or sequence:

In [ ]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
data[['two']]

,two
Ohio,1
Colorado,5
Utah,9
New York,13


In [ ]:
data[['three', 'one']]

,three,one
Ohio,0,0
Colorado,6,0
Utah,10,8
New York,14,12


Indexing like this has a few special cases. First, slicing or selecting data with a boolean
array:

In [ ]:
data[:'Utah']

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11


In [ ]:
data[:3]

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11


In [ ]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


The row selection syntax `data[:2]` is provided as a convenience. Passing a single element or a list to the `[]` operator selects columns.

Another use case is in indexing with a `boolean` `DataFrame`, such as one produced by a
scalar comparison:

In [ ]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [ ]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


This makes DataFrame syntactically more like a two-dimensional `NumPy` array in
this particular case.

### Selection with loc and iloc

For DataFrame label-indexing on the rows, the special indexing operators
`loc` and `iloc`. They enable you to select a subset of the rows and columns from a
`DataFrame` with `NumPy`-like notation using either axis labels *(loc)* or integers
*(iloc)*.

As a preliminary example, let’s select a single row and multiple columns by label:

In [ ]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
data.loc['New York', "one":"three"]

one      12
two      13
three    14
Name: New York, dtype: int64

In [ ]:
data.loc[:,'two':'three']

,two,three
Ohio,1,2
Colorado,5,6
Utah,9,10
New York,13,14


We’ll then perform some similar selections with integers using iloc:

In [ ]:
data.iloc[0:3, 1:]

,two,four
Ohio,1,3
Colorado,5,7
Utah,9,11


In [ ]:
data.iloc[2:4, [3, 0, 1]]

NameError: ignored

In [ ]:
data.iloc[:,[3, 0, 1]]

,four,one,two
Ohio,3,0,1
Colorado,7,4,5
Utah,11,8,9
New York,15,12,13


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, Ohio to New York
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   one     4 non-null      int64
 1   two     4 non-null      int64
 2   three   4 non-null      int64
 3   four    4 non-null      int64
dtypes: int64(4)
memory usage: 332.0+ bytes


In [ ]:
data.iloc['Colorado']

TypeError: ignored

In [ ]:
data.iloc[[1, 2], [3, 0, 1]]

NameError: ignored

Both indexing functions work with slices in addition to single labels or lists of labels:

In [ ]:
data.loc['Colorado':'Utah', :'three']

,one,two,three
Colorado,0,5,6
Utah,8,9,10


In [ ]:
data.iloc[:, :3][data["three"] > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [ ]:
train_d = int(len(data) * .75)
train_d

3

In [ ]:
data.iloc[:train_d]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11


In [ ]:
data.iloc[train_d:]

,one,two,three,four
New York,12,13,14,15


In [ ]:
data["one"].mean()

6.0

So there are many ways to select and rearrange the data contained in a pandas object.